### Load packages

In [1]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
import os
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
from nltk.stem import WordNetLemmatizer
import re
import string
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.metrics import confusion_matrix
import seaborn as sns
from imblearn.over_sampling import SMOTE
import gensim 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
import logging
from numpy import loadtxt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from time import strptime
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.io import push_notebook, output_notebook
nltk.download('stopwords')
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import gzip
import matplotlib.pyplot as plt
import nltk
import pickle
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
import unicodedata

c:\users\ae07525\python\python37-32\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ae07525\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) > 2]    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"other", "", text)
    text = re.sub(r"-", "", text)
    text = re.sub(r":", "", text)
    text = re.sub(r"an", "", text)

    ## Stemming
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lem_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lem_words)


    return text

### Data preprocessing and cleaning text

In [2]:
def data_preprocessing(df):


    df['Date']=pd.to_datetime(df['Submission Date'])
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year

    df['Month']=df['Month'].astype(str)
    print ('\n')
    
    #print ("Number of overall words in body",df['Comments'].apply(lambda x: len(x.split(' '))).sum())
    print ('\n')


    look_up = {'1': 'Jan', '2': 'Feb', '3': 'Mar', '4':
           'Apr', '5': 'May', '6': 'Jun', '7': 'Jul', '8': 'Aug', '9': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}

    df['Month'] = df['Month'].apply(lambda x: look_up[x])
    conditions = [(df['Overall Rating'] >3)  ,(df['Overall Rating'] ==3) ,(df['Overall Rating']< 3)]
    choices = ['positive', 'neutral', 'negative']
  
    
    df['feeling'] = np.select(conditions, choices, default='black')
    df["label"]=df["feeling"].astype('category').cat.codes
    df.Comments = df.Comments.str.replace('\d+', '')
    df['Comments'] = df['Comments'].astype(str)

    print ("Distribution of actual ",df.groupby('feeling').size())
    df['Comments'] = df['Comments'].str.lower()
    df['Comments'] = df['Comments'].str.strip()
    df = df[df['Comments'] != '??']
    df = df[df['Comments'] != '']
        
    df['comment'] = df['Comments'].apply(clean_text)
    #print ("Number of overall words after cleaning",df['comment'].apply(lambda x: len(x.split(' '))).sum())

    print(df['comment'][0:2])
    
    return df



## Set HyperParameter for Random Forest

In [ ]:


n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


### Train three classification algorithms random forest,  naive bayes and logistic regression and save in Pickel

In [ ]:

def train_model(random_grid , training_data):

    rf= RandomForestClassifier()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 3, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    rfc_hyper_search = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),
                ('clf', rf_random),])
    rfc_hyper_search.fit( training_data.comment,  training_data.feeling)
    
    print ("Save RF Model in pickel")
    pkl_filename = "pickle_model_RF.pkl"  
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(rfc_hyper_search, file)   
    my_tags = ['negative','neutral','positive']
    y_pred_HP_RF= rfc_hyper_search.predict( training_data.comment)
    print('accuracy in Random Forest %s' % accuracy_score(  training_data.feeling, y_pred_HP_RF))
    print ('Confution Matrix in Random Forest')
    print(classification_report( training_data.feeling, y_pred_HP_RF,target_names=my_tags))
    print (confusion_matrix(training_data.feeling, y_pred_HP_RF, my_tags))
    training_data["Sentimin_feeling_RF"]=y_pred_HP_RF

    my_tags = ['negative','neutral','positive']
    nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
    nb.fit(training_data.comment,  training_data.feeling)    
    print ("Save Naive Base in pickel")
    pkl_filename_nb = "pickle_model_NB.pkl"  
    with open(pkl_filename_nb, 'wb') as file:  
        pickle.dump(nb, file)
    y_pred_nb= nb.predict( training_data.comment)
    print('accuracy Naive Base%s' % accuracy_score(  training_data.feeling, y_pred_nb))
    print ('Confution Matrix Naive Base')
    print(classification_report( training_data.feeling,  y_pred_nb,target_names=my_tags))
    print (confusion_matrix(training_data.feeling,  y_pred_nb, my_tags))
    training_data["Sentimin_feeling_NB"]=y_pred_nb
   
           
    logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
    logreg.fit(training_data.comment,  training_data.feeling)
    
    print ("Save Logistic Regression in pickel")
    pkl_filename_lg = "pickle_model_LG.pkl"  
    with open(pkl_filename_lg, 'wb') as file:  
        pickle.dump(logreg, file)  
                 
    y_pred_lg= logreg.predict( training_data.comment)
    print('accuracy Logestic Regression %s' % accuracy_score(  training_data.feeling, y_pred_lg))
    print ('Confution Matrix Logistic Regression')
    print(classification_report( training_data.feeling,  y_pred_lg,target_names=my_tags))
    print (confusion_matrix(training_data.feeling,  y_pred_lg, my_tags))
    training_data["Sentimin_feeling_LG"]=y_pred_lg
    print ( training_data[0:3])
    
    return training_data, rfc_hyper_search, nb, logreg
    

### load models from pickel

In [4]:
print ("Load Random Forest Model")    
pickle_in_RF = open("pickle_model_RF.pkl","rb")
load_model_sentiment_RF = pickle.load(pickle_in_RF)

print ("Load Naive Base Model ")    
pickle_in_NB = open("pickle_model_NB.pkl","rb")
load_model_sentiment_NB = pickle.load(pickle_in_NB) 


print ("Load Logistic Regression")    
pickle_in_LG = open("pickle_model_LG.pkl","rb")
load_model_sentiment_LG = pickle.load(pickle_in_LG) 

Load Random Forest Model
Load Naive Base Model 
Load Logistic Regression



## Precicting new comment feeling by sentiment classification



#### Model was trained by Opinoin lab from Jan 2017 to Oct 2017

In [31]:
def prediction_evaluation(new_review,load_model_sentiment_RF,load_model_sentiment_NB,load_model_sentiment_LG):
    
    my_tags = ['negative','neutral','positive']
    sentiment_random_forest= load_model_sentiment_RF.predict(new_review.comment)
    print('accuracy test Random Forest %s' % accuracy_score( new_review.feeling, sentiment_random_forest))
    print ('Confution Matrix test Random Forest')
    print(classification_report(new_review.feeling, sentiment_random_forest,target_names=my_tags))
    new_review["sentiment_random_forest"]=sentiment_random_forest
    print ("Distribution of ",new_review.groupby('sentiment_random_forest').size())
    print (new_review[0:3])
    print ("Load Naive Base Model ")    
    new_sentiment_NB= load_model_sentiment_NB.predict(new_review.comment)

    print('accuracy test Naive Base %s' % accuracy_score( new_review.feeling, new_sentiment_NB))
    print ('Confution Matrix test Random Forest')
    print(classification_report(new_review.feeling, new_sentiment_NB,target_names=my_tags))

    new_review["Sentimin_NiveBase"]=new_sentiment_NB
    print ("Distribution of ",new_review.groupby('Sentimin_NiveBase').size())

    new_sentiment_LG= load_model_sentiment_LG.predict(new_review.comment)

    print('accuracy test LogisticRegression %s' % accuracy_score( new_review.feeling, new_sentiment_LG))
    print ('Confution Matrix test Random Forest')
    print(classification_report(new_review.feeling, new_sentiment_LG,target_names=my_tags))

    new_review["Sentimin_Logistic_Regression"]=new_sentiment_LG
    print ("Distribution of ",new_review.groupby('Sentimin_Logistic_Regression').size())

    new_review['majority']= np.where((new_review['sentiment_random_forest'].str.contains('negative'))&( new_review['Sentimin_Logistic_Regression'].str.contains ('positive|neutral'))&( new_review['Sentimin_NiveBase'].str.contains('positive|neutral')),'neutral', new_review.sentiment_random_forest)
    new_review['majority']= np.where((new_review['sentiment_random_forest'] == 'positive')&(new_review['Sentimin_Logistic_Regression'] == 'negative')&(new_review['Sentimin_NiveBase'] == 'negative'),'neutral',new_review['majority'])
    
    print (new_review[0:3])  
    print ("Saving new_senmtimet_label in CSV format")
    new_review.to_csv('C://Users//ae07525//Desktop//New_folder//New_text_Sentiment_majority.csv')
    
    return new_review
    
    

### Upload  dataset opinion lab from Jan 2017 to Feb 2019

In [34]:
opinionlab2017182019=pd.read_csv("C://Users//ae07525//PycharmProjects//bachtime//formatted_Feb25.csv", error_bad_lines=False,encoding='utf-8')

opinionlab2017=pd.read_csv("OpinionLab_012017_122017.csv", error_bad_lines=False, encoding="ISO-8859-1")
opinionlab=pd.concat([opinionlab2017,opinionlab2017182019])

### Remove some noisy comments that is similar to Doc2Vec

In [35]:
print (opinionlab.shape)
opinionlab.dropna()
# drop duplicates
opinionlab.drop_duplicates(subset=None, keep='first', inplace=True)
#drop non ascii chars

opinionlab = opinionlab[opinionlab.Comments.str.count(' ')>1]

opinionlab=opinionlab.dropna()
opinionlab=opinionlab[~opinionlab.Comments.str.contains("Farhat Deeba")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("farhat deeba")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("ï¼ xì ¼ê²½ì xë ï¼")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("ï¼")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("Farhat Deeba")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("Ã¯Â¼Â‚xÃ¬Â‹ÂœÃªÂ°Â")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("¼ê²½ìƒ‰Â·ë")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("xxzxzdrddxxxxxxxf")]
opinionlab=opinionlab[~opinionlab.Comments.str.contains("¼ê²½ì")] 
opinionlab=opinionlab[~opinionlab.Comments.str.contains("Facebook | Facebook")] 
opinionlab=opinionlab[~opinionlab.Comments.str.contains("farhat deeba xx bird eye circle stony brook ny xxxxx usa")] 


print (opinionlab.shape)

(149900, 9)
(89045, 9)


### Read Functions

In [36]:
opl=data_preprocessing(opinionlab)






Distribution of actual  feeling
negative    53166
neutral     15966
positive    19913
dtype: int64
42256           safari comment card test
42546    custom variable formatting test
Name: comment, dtype: object


### Split data as training dataset and test dataset

### Split data for 2017 and 2018 for trining dataset and 2019 for test dataset

In [37]:

training_data= opl[(opl['Date'] < '2019-01-01')]
test_data= opl[(opl['Date'] > '2019-01-01')]

test_data.shape, training_data.shape, pd.unique(training_data.Year), pd.unique(test_data.Year)

((15415, 15),
 (73628, 15),
 array([2017, 2018], dtype=int64),
 array([2019], dtype=int64))

In [38]:
print ("Month and years in training datasetin OPL",pd.unique(training_data.Month),pd.unique(training_data.Year))
print ("Month and year in test dataset in OPL", pd.unique(test_data.Month))

month and year in training datasetin OPL ['Dec' 'Jan' 'Feb' 'Mar' 'Apr' 'May' 'Jun' 'Jul' 'Aug' 'Sep' 'Oct' 'Nov'] [2017 2018]
Month and years in test dataset in OPL ['Jan' 'Feb']


### Call training function 

In [44]:
 training_data, rfc_hyper_search, nb, logreg=train_model(random_grid , training_data)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed: 18.9min remaining:  5.4min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 26.0min finished


Save RF Model in pickel
accuracy in Random Forest 0.6804962817203751
Confution Matrix in Random Forest
              precision    recall  f1-score   support

    negative       0.66      0.99      0.79     45764
     neutral       0.89      0.01      0.02     14315
    positive       0.85      0.42      0.57     18183

   micro avg       0.68      0.68      0.68     78262
   macro avg       0.80      0.48      0.46     78262
weighted avg       0.75      0.68      0.60     78262

[[45373     5   386]
 [13184   170   961]
 [10453    16  7714]]
Save Naive Base in pickel
accuracy Naive Base0.6660320462037771
Confution Matrix Naive Base
              precision    recall  f1-score   support

    negative       0.65      0.99      0.78     45764
     neutral       0.66      0.03      0.06     14315
    positive       0.86      0.35      0.50     18183

   micro avg       0.67      0.67      0.67     78262
   macro avg       0.72      0.46      0.45     78262
weighted avg       0.70      0.67 

c:\users\ae07525\python\python37-32\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Save Logistic Regression in pickel
accuracy Logestic Regression 0.78721474023153
Confution Matrix Logistic Regression
              precision    recall  f1-score   support

    negative       0.82      0.92      0.87     45764
     neutral       0.65      0.44      0.52     14315
    positive       0.77      0.72      0.74     18183

   micro avg       0.79      0.79      0.79     78262
   macro avg       0.75      0.69      0.71     78262
weighted avg       0.78      0.79      0.78     78262

[[42310  1726  1728]
 [ 5882  6253  2180]
 [ 3533  1604 13046]]
            Source      Submission Date                         Comments  \
42256  Opinion Lab  12/18/2017 04:06 PM         safari comment card test   
42546  Opinion Lab  12/21/2017 06:20 AM  custom variable formatting test   
42548  Opinion Lab  12/21/2017 06:22 AM    right formatting confirmation   

      Browser Name OS Name  Overall Rating   Device Type  \
42256       Safari    OS X               5       Desktop   
42546       

In [39]:
### Test the model in comment that is generated in 2019  

In [40]:

test_data=prediction_evaluation(test_data,load_model_sentiment_RF,load_model_sentiment_NB,load_model_sentiment_LG)

accuracy test Random Forest 0.6864742134284788
Confution Matrix test Random Forest
              precision    recall  f1-score   support

    negative       0.68      0.99      0.81      9848
     neutral       0.32      0.00      0.01      2716
    positive       0.74      0.30      0.42      2851

   micro avg       0.69      0.69      0.69     15415
   macro avg       0.58      0.43      0.41     15415
weighted avg       0.63      0.69      0.60     15415

Distribution of  sentiment_random_forest
negative    14253
neutral        25
positive     1137
dtype: int64
            Source      Submission Date  \
32193  Opinion Lab  01/01/2019 12:04 AM   
32197  Opinion Lab  01/01/2019 12:51 AM   
32198  Opinion Lab  01/01/2019 01:12 AM   

                                                Comments  \
32193             needs to show due date on payment page   
32197  please add pending $ totals to the app. would ...   
32198  the app is nice but doesn't work more often th...   

              